# 載入所需套件

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from urllib.request import urlretrieve
from concurrent import futures
from tqdm import tqdm

# 定義爬取個別Yahoo電影院線片資訊細節函數

In [2]:
def details_crawler(link_list):
    link=link_list[0]
    i=link_list[1]
    r=requests.get(link,stream=True)
    s=BeautifulSoup(r.text,'html.parser')
    movie_intro_info_r=s.select('div.movie_intro_info_r')[0]
    
    return i,movie_intro_info_r    

# 定義爬取Yahoo電影院線片排名資訊函數

In [3]:
def yahoo_movie_crawler(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')

    movies=soup.select('ul.release_list>li')
    cnames=[e.select('a')[1].text.replace('\n','').replace(' ','') for e in movies]
    links=[e.select('a')[1]['href'] for e in movies] 
    popularitys=[e.select('div.leveltext>span')[0].text for e in movies]
    points=[e.select('span.count')[0]['data-num'] for e in movies]
    texts=[e.select('div.release_text')[0].text.replace('\n','').replace('\r','').replace(' ','') for e in movies]
    dates=[e.select('div.release_movie_time')[0].text.split(' ： ')[1] for e in movies]
    imgs=[e.select('img')[0]['src'] for e in movies]
    #處理可能英文電影名遺漏值
    enames=[]
    for e in movies:
        if e.select('a')[2].text.replace('\n','').replace('  ','')=='':
            enames.append(np.nan)
        else:
            enames.append(e.select('a')[2].text.replace('\n','').replace('  ',''))
            
    directors=[0]*len(links)
    actors=[0]*len(links)
    periods=[0]*len(links)
    companys=[0]*len(links)
    imdbs=[0]*len(links)
            
    link_list=[]
    for i in range(len(links)):
        link_list.append([links[i],i])       
    
     #進入每個連結爬取(執行thread層級的非同步任務)
    with futures.ThreadPoolExecutor(max_workers=8) as executor:
        results=list(executor.map(details_crawler,link_list))

        for future in results:
            i,movie_intro_info_r=future
            directors[i]=movie_intro_info_r.select('div.movie_intro_list')[0].text.replace('\n','').replace(' ','')
            periods[i]=movie_intro_info_r.select('span')[1].text.split('：')[1]
            companys[i]=movie_intro_info_r.select('span')[2].text.split('：')[1]
            #處理可能演員遺漏值
            if movie_intro_info_r.select('div.movie_intro_list')[1].text.replace('\n','').replace(' ','')=='':
                actors[i]=np.nan
            else:
                actors[i]=movie_intro_info_r.select('div.movie_intro_list')[1].text.replace('\n','').replace(' ','')
            #處理可能IMDb分數遺漏值
            if movie_intro_info_r.select('span')[3].text.split('：')[1]=='':
                imdbs[i]=np.nan
            else:
                imdbs[i]=movie_intro_info_r.select('span')[3].text.split('：')[1]
    
    df=pd.DataFrame({
        '中文電影名':cnames,
        '英文電影名':enames,
        '上映日':dates,
        '導演':directors,
        '演員':actors,
        '片長':periods,
        'IMDb分數':imdbs,
        '期待度':popularitys,
        '滿意度':points,
        '介紹':texts,
        '網址':links,
        '電影劇照':imgs
    })        
    
    return df

# 定義爬取Yahoo電影院線片排名頁數函數

In [4]:
def page_function(page):
    dfs=[]
    
    for i in tqdm(range(page)):
        url='https://movies.yahoo.com.tw/movie_intheaters.html?page={}'.format(i+1)
        d=yahoo_movie_crawler(url)
        dfs.append(d)
    
    df=pd.concat(dfs,ignore_index=True)
    
    return df

# 指定總頁數表格呈現

In [5]:
df=page_function(3)
df.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.78s/it]


,中文電影名,英文電影名,上映日,導演,演員,片長,IMDb分數,期待度,滿意度,介紹,網址,電影劇照
0,音速小子,Sonic the Hedgehog,2020-02-21,傑夫富勒(JeffFowler),金凱瑞(JimCarrey)、詹姆斯馬斯登(JamesMarsden)、班許瓦茲(BenSc...,01時39分,6.9,69%,4.1,★《死侍》、《玩命關頭》幕後團隊最新力作 ★智慧大反派金凱瑞端出高科技武器PK全新超級快英雄...,https://movies.yahoo.com.tw/movieinfo_main/%E9...,https://movies.yahoo.com.tw/x/r/w420/i/o/produ...
1,李察朱威爾事件,Richard Jewell,2020-02-21,克林伊斯威特(ClintEastwood),山姆洛克威爾(SamRockwell)、奧莉維亞魏爾德(OliviaWilde)、喬漢姆(J...,02時11分,7.5,92%,4.3,由克林伊斯威特執導，根據真實事件改編，《李察朱威爾事件》一片描述當不實的報導被當作事實時，真...,https://movies.yahoo.com.tw/movieinfo_main/%E6...,https://movies.yahoo.com.tw/x/r/w420/i/o/produ...
2,絕命大平台,The Platform,2020-02-21,高德加斯泰盧烏魯提亞(GalderGaztelu-Urrutia),伊萬馬薩格(IvanMassagué)、安東妮亞聖胡安(AntoniaSanJuan)、佐里...,01時34分,7.2,79%,3.5,★挖掘人性直逼《奪魂鋸》，禁閉設定媲美《異次元殺陣》★反烏托邦宛如《摩天樓》，為了自由，你能...,https://movies.yahoo.com.tw/movieinfo_main/%E7...,https://movies.yahoo.com.tw/x/r/w420/i/o/produ...
3,陰櫥,The Closet,2020-02-21,金光斌,河正宇(Jung-wooHa)、金南佶,01時38分,6.3,86%,4.0,★《白頭山：半島浩劫》製片團隊最新恐怖懼作★《白頭山：半島浩劫》河正宇×《熱血司祭》金南佶精...,https://movies.yahoo.com.tw/movieinfo_main/%E9...,https://movies.yahoo.com.tw/x/r/w420/i/o/produ...
4,咒怨電影版,Ju-On,2020-02-21,清水崇(TakashiShimizu),奧菜惠、伊東美咲、上原美佐、市川由衣,01時32分,6.7,79%,4.1,★繼《七夜怪談》七夜怪談中田秀夫後，日本恐怖大師清水祟自編自導的一鳴驚人之作。★J-Horr...,https://movies.yahoo.com.tw/movieinfo_main/%E5...,https://movies.yahoo.com.tw/x/r/w420/i/o/produ...


# 抓取Yahoo電影院線片劇照

In [6]:
cnames=df['中文電影名'].values.tolist()
imgs=df['電影劇照'].values.tolist()

directory='Yahoo電影院線片劇照'
if not os.path.isdir(directory):
    os.makedirs(directory)

for cname,img in zip(cnames[:10],imgs[:10]):    
    print(cname)
    urlretrieve(img,directory+'/{}.jpg'.format(cname))

音速小子
李察朱威爾事件
絕命大平台
陰櫥
咒怨電影版
少年阿罕默德
閃閃小超人電影版
吹哨奇案
悲慘世界
電影音效傳奇：好萊塢之聲
